# 数据准备

In [1]:
#导入包与读取数据
import numpy as np
import pandas as pd
import statsmodels.stats.proportion as ssp

data = pd.read_csv(r'D:\pythonstudy\game2.csv')
print(data.head(5))
print(data.info())

#字段解释
#version为版本
#sum_gamerounds为玩家玩的回数
#retention_1为次日留存率
#retention_2为七日留存率

   userid  version  sum_gamerounds  retention_1  retention_7
0     116  gate_30               3        False        False
1     337  gate_30              38         True        False
2     377  gate_40             165         True        False
3     483  gate_40               1        False        False
4     488  gate_40             179         True         True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB
None


In [2]:
#数据清洗：空值 重复值 异常值
print('data.isnull:\n',data.isnull().sum())
print('data.duplicated:',data.duplicated().sum())

print('版本：', data.version.unique())
print('次日留存率：', data.retention_1.unique())
print('七日留存率：', data.retention_7.unique())

#该数据比较干净，没有需要清洗的值

data.isnull:
 userid            0
version           0
sum_gamerounds    0
retention_1       0
retention_7       0
dtype: int64
data.duplicated: 0
版本： ['gate_30' 'gate_40']
次日留存率： [False  True]
七日留存率： [False  True]


# 假设检验
- 确定样本分布类型进行相关假设检验
- 验证并得出结论

In [3]:
#提取分析所需数据
retention = data.groupby('version').agg({'userid':'count','retention_1':'sum','retention_7':'sum'})
retention

,userid,retention_1,retention_7
version,,,
gate_30,44700,20034,8502
gate_40,45489,20119,8279


1. 假设检验：原假设和备择假设
设gate_30概率为P1，gate_40概率为P2，可得假设：
原假设H0：P1≥P2
备择假设Ha：P1<P2
2. 分布类型、检验类型和显著性水平
大样本，采用Z检验。显著性水平α取0.05

In [4]:
retention_rate = pd.DataFrame(index = ['gate_30','gate_40'],columns = ['次日留存率','七日留存率'])
for i in range(0,2):
    retention_rate.iloc[i,1] = retention.iloc[i,2] / retention.iloc[i,0]
    retention_rate.iloc[i,0] = retention.iloc[i,1] / retention.iloc[i,0] 
retention_rate

,次日留存率,七日留存率
gate_30,0.448188,0.190201
gate_40,0.442283,0.182


In [5]:
# 次日留存率的检验
z_score, p_value = ssp.proportions_ztest(count = retention.retention_1, nobs = retention.userid, alternative='smaller')
print("Z值为：", z_score)
print("P值为：", p_value)

alpha = 0.05
if p_value < alpha:
    print(f"\n由于 P 值 ({p_value:.4f}) 小于显著性水平 ({alpha})，我们拒绝原假设。")
    print("结论：结论：两个版本的次日留存率存在显著差异，gate_30的留存率更高。")
else:
    print(f"\n由于 P 值 ({p_value:.4f}) 大于显著性水平 ({alpha})，我们无法拒绝原假设。")
    print("结论：两个版本的次日留存率没有显著差异。")

Z值为： 1.7840862247974725
P值为： 0.9627951723515404

由于 P 值 (0.9628) 大于显著性水平 (0.05)，我们无法拒绝原假设。
结论：两个版本的次日留存率没有显著差异。


In [6]:
# 七日留存率
z_score, p_value = ssp.proportions_ztest(count = retention.retention_7, nobs = retention.userid, alternative='smaller')
print("Z值为：", z_score)
print("P值为：", p_value)

alpha = 0.05
if p_value < alpha:
    print(f"\n由于 P 值 ({p_value:.4f}) 小于显著性水平 ({alpha})，我们拒绝原假设。")
    print("结论：两个版本的七日留存率存在显著差异，gate_30的留存率更高。")
else:
    print(f"\n由于 P 值 ({p_value:.4f}) 大于显著性水平 ({alpha})，我们无法拒绝原假设。")
    print("结论：两个版本的七日留存率没有显著差异。")

Z值为： 3.164358912748191
P值为： 0.9992228750121929

由于 P 值 (0.9992) 大于显著性水平 (0.05)，我们无法拒绝原假设。
结论：两个版本的七日留存率没有显著差异。
